# Event sourcing

MapleDB uses `eventstore` instead of `active records`. This means that instead of writing directly to the tables, the client will write the changes it wants to perform to an extra table, the `eventstore`, to keep track of every cahnges. This allows not only to keep track of these changes, but also to revert the database to any point of time if needed.

While event sourcing is very good method to avoid data-loss, it does not protect a database against hardware failure. Keep this in mind when setting up your server to host MapleDB.



In [1]:
import mdb

client = mdb.MDBClient(hostname='localhost',
                       username='postgres',
                       password='',
                       database='mdb')

es = client.get('eventstore')
es

,event,data,uuid,id,type,timestamp
0,create,"{'smiles': 'DEF', 'molecule_type_id': '47313a9...",2e451d97-3e56-4798-b493-fc3574a7b13f,23,molecule,2020-04-01 09:23:45.794141
1,create,"{'x': [0.0, 0.10101010101010101, 0.20202020202...",8317394a-45f6-459b-940a-9601aef28cf4,22,xy_data,2020-04-01 09:21:17.498280
2,create,{'name': 'stupid_units'},a6e79a81-c91b-49d6-a620-d8019b805ef6,21,data_unit,2020-04-01 09:21:17.435500
3,create,"{'notes': '', 'metadata': {'param1': 100, 'kno...",6e28b0ba-184b-4f0d-b400-c13d3b2909a6,20,experiment,2020-04-01 09:21:17.363519
4,create,"{'make': 'Super NMR', 'name': 'Mad NMR', 'mode...",247f4f20-903a-430a-8063-7541a37c42dc,19,experiment_machine,2020-04-01 09:21:17.291960
5,create,{'name': 'NMR'},59ed84db-0671-46cf-b22c-da8a83115dbf,18,experiment_type,2020-04-01 09:21:17.111681
6,update,{'notes': ' # Synthesis of ABC Everything wen...,fe4d8183-5140-475b-b0e7-811bcc2db502,17,synthesis,2020-04-01 09:21:16.816755
7,create,"{'hid': 'MAD_2020-04-01_0', 'xdl': '<xdl><reci...",fe4d8183-5140-475b-b0e7-811bcc2db502,16,synthesis,2020-04-01 09:21:16.664410
8,update,"{'cas': '13283-31-3', 'cid': 6331.0, 'inchi': ...",1338c13b-8ec0-455d-806e-549256c5e2b4,14,molecule,2020-04-01 09:21:16.419759
9,update,"{'cas': None, 'cid': None, 'inchi': None, 'smi...",0b0a88df-4d69-4eea-88a9-8c3bceeeeba0,15,molecule,2020-04-01 09:21:16.419759


## Rollback

It is possible to `rollback` the database to revert to any point of time. To do so, the rollback method can be used:

In [2]:
client.rollback(before=es.at[3, 'timestamp'].to_pydatetime())

In [3]:
client.get('eventstore').sort_values(by=['id'])

,event,data,id,uuid,type,timestamp
61,create,"{'name': 'Mad Lab', 'short_name': 'MAD'}",1,5541b619-c181-4647-98b6-79516ee24800,lab,2020-04-01 09:20:25.145175
60,create,"{'make': 'MadChem', 'name': 'Mad Machine Doing...",2,59aa61d2-bb84-43ea-a58c-11325dfefe57,synthesis_machine,2020-04-01 09:20:25.327287
59,create,{'name': 'fragment'},3,4dcfb667-5c9c-4d4f-a123-97cd8bee0513,molecule_type,2020-04-01 09:20:25.431244
58,create,"{'smiles': 'A', 'molecule_type_id': '4dcfb667-...",4,0b0a88df-4d69-4eea-88a9-8c3bceeeeba0,molecule,2020-04-01 09:20:25.475240
57,create,"{'cas': '13283-31-3', 'cid': 6331, 'inchi': 'I...",5,1338c13b-8ec0-455d-806e-549256c5e2b4,molecule,2020-04-01 09:20:41.715173
...,...,...,...,...,...,...
19,create,"{'hid': 'MAD_2020-04-01_0', 'xdl': '<xdl><reci...",58,fe4d8183-5140-475b-b0e7-811bcc2db502,synthesis,2020-04-01 09:24:29.687648
20,update,{'notes': ' # Synthesis of ABC Everything wen...,59,fe4d8183-5140-475b-b0e7-811bcc2db502,synthesis,2020-04-01 09:24:29.687648
21,create,{'name': 'NMR'},60,59ed84db-0671-46cf-b22c-da8a83115dbf,experiment_type,2020-04-01 09:24:29.687648
22,create,"{'make': 'Super NMR', 'name': 'Mad NMR', 'mode...",61,247f4f20-903a-430a-8063-7541a37c42dc,experiment_machine,2020-04-01 09:24:29.687648


We can see here from the event `rollback` everything is deleted and then the event are sequentially repeated until the we reach the given timestamp.